In [189]:
import os
import pickle
import cv2
import numpy as np
import math
import time
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot

### Set up camera and robot, load pre-trained SSD model for COCO

In [2]:
model = ObjectDetector('../Notebooks/object_following/ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300)
robot = Robot()

In [190]:
# Load COCO labels
filename = "coco_labels.dat"
filehandler = open(filename, 'rb')
COCO_labels = pickle.load(filehandler)

# Load camera calibration data for undistort
filename = "calibration.dat"
filehandler = open(filename, 'rb')
camera_cal = pickle.load(filehandler)
mtx = camera_cal['mtx']
dist = camera_cal['dist']
f_u = mtx[0,0]   # focal center coordinates
f_v = mtx[1,1]
focal_center = np.array([f_u, f_v])

# Mapping between set_motor "speed" and measured wheel angular velocity "omega"
# for 0.1 second motor running time
wheel_calibration = {
    "speed": [0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    "omega": [0.0, 3.85, 9.23, 15.0, 25.8, 29.2, 35.4]
}

# Open Image Widget
image_widget = widgets.Image(format='jpeg', width=300, height=300)
width = int(image_widget.width)
height = int(image_widget.height)

BLUE = (255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)

diag_dir = 'diagnostics'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(diag_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


In [236]:
start_bbox_TV = [(102, 90), (161, 134)]

pos1_bbox_TV = [(102, 90), (164, 135)]
pos1_bbox_stool = [(168, 123), (200, 182)]

pos2_bbox_TV = [(99, 79), (170, 128)]
pos2_bbox_stool = [(171, 115), (211, 183)]

pos2left_bbox_TV = [(136, 64), (218, 132)]
pos2left_bbox_stool = [(215, 96), (283, 187)]

pos2right_bbox_TV = [(51, 69), (134, 131)]
pos2right_bbox_stool = [(131, 122), (164, 184)]

pos3_bbox_TV = [(90, 72), (172, 127)]
pos3_bbox_stool = [(172, 107), (219, 189)]

# Bbox for TV and stool at various camera poses
# For (0,0,0) to (1,0,0)
beg_bbox_TV = [(72, 92), (136, 137)]  # Pose (0,0,0)
mid_bbox_TV = [(33, 69), (114, 124)]  # Pose (0.5,0,0)
end_bbox_stool = [(175, 84), (237, 186)]   # Pose (1.0,0,0)

# Bbox for 2nd stool at various camera poses
# For (1,0,0) to (2,0,pi)
end_bbox_horse = [(147, 103), (247, 201)]
mid_bbox_stool_2 = [(163, 80), (263, 185)]   # midpoint to Pose (1,2,0)

In [288]:
def undistort(img, mtx, dist, crop=False):
    """Undistort camera image based on calibration data"""
    h,w = img.shape[:2]
    # print (h,w)
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image (optional)
    if crop:
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
    return dst

def draw_bbox(img, width, height, bbox, color, line_width):
    bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                  (int(width * bbox[2]), int(height * bbox[3]))]
    cv2.rectangle(img, bbox_pixel[0], bbox_pixel[1], color, line_width)
    return bbox_pixel

def display_detected(img, detections, width, height, debug=False):
    """ put blue bounding boxes on detected objects on image """
    
    for det in detections[0]:
        label = COCO_labels[det['label']]
        bbox = det['bbox']
        bbox_pixel = draw_bbox(img, width, height, bbox, BLUE, 1)
        if debug:
            print(label,det['label'], bbox_pixel)
    return

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)

def object_center(object_features):
    """Computes the center x, y coordinates of the object using its feature points"""
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
    center_x = (UL_fp_x + LR_fp_x) / 2.0 - 0.5
    center_y = (UL_fp_y + LR_fp_y) / 2.0 - 0.5
    return np.array([center_x, center_y])
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections, debug= False):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if debug:
            print(center)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def create_feature_pts(bbox):
    """ Return 4 feature points from a bounding box """
    # extract bounding x and y coordinates
    UL_fp, LR_fp = bbox
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
   
    # return UL(upper left), UR(upper right), LR(lower right), LL(lower left) feature points
    return [(UL_fp_x, UL_fp_y), (LR_fp_x, UL_fp_y), (LR_fp_x, LR_fp_y), (UL_fp_x, LR_fp_y)]

def generate_ref_obj_features(ref_obj, img, detections, width, height, debug=False):
    """ Find reference object in detected objects and generate 4 feature points """
    matching_detections = [d for d in detections[0] if d['label'] == ref_obj]

    # get detection closest to center of field of view and draw it (if model detects multiple
    # reference objects)
    det = closest_detection(matching_detections)

    if det is not None:
        bbox = det['bbox']
        # bound reference object in green
        draw_bbox(img, width, height, bbox, GREEN, 1)
        # convert to pixel units
        bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                       (int(width * bbox[2]), int(height * bbox[3]))]
        return create_feature_pts(bbox_pixel) # return 4 feature points of bounding box
    else:
        return None
    
def fp_error(current, goal):
    """ Return error of a feature point """
    current_x, current_y = current
    goal_x, goal_y = goal
    # Use Corke's convention of (p*-p) --> gain is a +ve number
    return (goal_x-current_x, goal_y-current_y)  

def features_error(currentFeatures, goalFeatures):
    """ Return feature points error vector """
    UL_current, UR_current, LR_current, LL_current = currentFeatures 
    UL_goal, UR_goal, LR_goal, LL_goal = goalFeatures 
    
    # calculate error for each feature point
    UL_error = fp_error(UL_current, UL_goal)
    error_vector = np.asarray(UL_error)
    UR_error = fp_error(UR_current, UR_goal)
    error_vector = np.concatenate((error_vector, UR_error), axis=None)
    LR_error = fp_error(LR_current, LR_goal)
    error_vector = np.concatenate((error_vector, LR_error), axis=None)
    LL_error = fp_error(LL_current, LL_goal)
    error_vector = np.concatenate((error_vector, LL_error), axis=None)

    # return 1x8 error vector of UL, UR, LR, LL feature points
    return np.reshape(error_vector, (1,-1))

def image_jacobian(fp, mtx, depth):
    """ Generate image jacobiab L for a feature point """
    # focal lengths in pixel unit
    f_u = mtx[0,0]
    f_v = mtx[1,1]
    c_u = mtx[0,2]
    c_v = mtx[1,2]

    # Estimated distance of reference object (m)
    Z = depth

    # Calculate J of feature point
    u_raw, v_raw = fp
    u = u_raw - c_u
    v = v_raw - c_v    
    
    L = np.array([[-f_u/Z, 0, u/Z, u*v/f_u, -(f_u+u*u/f_u), v],
                  [0, -f_v/Z, v/Z, f_v+v*v/f_v, -u*v/f_v, -u]])
    return L

def robot2world(robot_orientation):
    """ calculate the robot jacobian """
    theta = robot_orientation
    
    return np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])

def control2robot(wheel_radius, axle_length):
    """ transform wheel speeds to robot motion in world frame """
    l = axle_length
    r = wheel_radius

    return np.array([[r/2, r/2],
                  [r/l, -r/l]])

def save_snapshot(img, directory, i):
    image_path = os.path.join(directory, 'detect'+str(i+1)+'.jpg')
    cv2.imwrite(image_path, img) 
    return

def omega2speed(in_val, mapping):
    """ Map wheel angular speed to motor speed setting based on a calibration mapping """
    
    if in_val < 0:
        sign = -1
        in_val = abs(in_val)
    else:
        sign = 1
        
    out_lower = 0
    in_lower = 0
    out_val = 0

    for i, in_upper in enumerate(mapping["omega"]):
        # print (i, in_upper)
        if in_val < in_upper:
            out_upper = mapping["speed"][i]
            out_val = out_lower + (in_val - in_lower)/(in_upper - in_lower) \
                *(out_upper-out_lower)
            # print("yes", out_val)
            break
        else:
            # print("no")
            out_lower = mapping["speed"][i]
            in_lower = in_upper
            
    if out_val is 0:
        print ("Input is too high!!!", in_val)
        out_val = 0
        
    return sign*out_val

def rotate_left_2wheels(speed, Rtime):
    
    robot.set_motors(-speed, speed)
    time.sleep(Rtime)
    robot.stop()
    
    return

In [247]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

# robot pose - may need to be updated
depth = 2.0
orientation = 0 

# robot parameters
wheel_radius = 0.0325
axle_length = 0.12

# Control loop parameters
gain = 1.35
Rtime = 0.1  # The loop run at 8fps --> 0.125s duration
num_iter = 5
interval = 2

no_motion = False  # Flag to disable motor (for debugging)
debug = True

# Use TV as reference object 
ref_obj = 72
goal_features = create_feature_pts(mid_bbox_TV)
print(ref_obj,COCO_labels[ref_obj], goal_features)


start = time.perf_counter()
# Run for fixed # of iterations
for i in range(num_iter):
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=True) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
    
    if obj_features is None:
        print("Reference object not detected!!!")
        continue
        
    # bound reference object's feature points at goal pose in red
    cv2.rectangle(image, goal_features[0], goal_features[2], RED, 2)
    error = features_error(obj_features,goal_features)
    error_norm = np.linalg.norm(error)
    
    if i%interval == 0 and debug:
        print(obj_features)
        print(error, error_norm)

    if np.linalg.norm(error) < 10:
        print ("Goal point achieved!!!")
        break
        
    T = control2robot(wheel_radius, axle_length) # wheel speeds to robot velocities transform
    H = robot2world(orientation)  # robot velocities to world frame transform
    
    # Image Jacobian based on 4 fp of reference object
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
    
    # implement w = gain * pinv(LHT).error
    J = np.dot(L,np.dot(H,T))
    inv_J = np.linalg.pinv(J)
    wheel_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
    robot_velocities = np.dot(T, wheel_velocities)
    w_r = omega2speed(wheel_velocities[0,0],wheel_calibration) 
    w_l = omega2speed(wheel_velocities[1,0],wheel_calibration) 
    
    if i%interval == 0 and debug:
        print("Iteration ", i)
        save_snapshot(image, diag_dir, i)
        print(wheel_velocities, (w_l, w_r))
        print(robot_velocities)
        print("Orientation:", orientation)
        print("Depth:", depth)
    
    if no_motion is False:
        robot.set_motors(w_l, w_r)
        time.sleep(Rtime)
        robot.stop()
    
    # Update robot's orientation
    translation_velocity = robot_velocities[0,0]
    angular_velocity = robot_velocities[1,0]
    orientation += Rtime*angular_velocity
    depth -= Rtime*translation_velocity*math.cos(orientation)
    
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image

image_widget.value = bgr8_to_jpeg(image)
robot.stop()

end = time.perf_counter()
print ("FPS: {:.1f}".format(num_iter/(end-start)))

72 laptop [(33, 69), (114, 69), (114, 124), (33, 124)]
couch 62 [(88, 112), (128, 182)]
clock 84 [(138, 64), (266, 186)]
Reference object not detected!!!
couch 62 [(88, 112), (127, 181)]
clock 84 [(138, 62), (266, 183)]
clock 84 [(162, 128), (169, 148)]
Reference object not detected!!!
couch 62 [(88, 111), (127, 181)]
clock 84 [(140, 68), (263, 186)]
clock 84 [(162, 128), (169, 148)]
Reference object not detected!!!
couch 62 [(88, 110), (127, 181)]
clock 84 [(137, 63), (266, 188)]
clock 84 [(161, 128), (169, 147)]
Reference object not detected!!!
couch 62 [(88, 116), (126, 181)]
clock 84 [(139, 68), (262, 185)]
clock 84 [(161, 128), (169, 148)]
Reference object not detected!!!
FPS: 8.0


In [107]:
robot.stop()

In [201]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

# robot pose - may need to be updated
depth = 1.3
orientation = 0 

# Control loop parameters
gain = 1.5
Rtime = 0.1  # The loop run at 8fps --> 0.125s duration
num_iter = 30
interval = 2

no_motion = False  # Flag to disable motor (for debugging)
debug = True

# Use stool as reference object
ref_obj = 62
goal_features = create_feature_pts(end_bbox_stool)
print(ref_obj,COCO_labels[ref_obj], goal_features)

start = time.perf_counter()
# Run for fixed # of iterations
for i in range(num_iter):
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=True) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
    
    if obj_features is None:
        print("Reference object not detected!!!")
        continue
        
    # bound reference object's feature points at goal pose in red
    cv2.rectangle(image, goal_features[0], goal_features[2], RED, 2)
    error = features_error(obj_features,goal_features)
    error_norm = np.linalg.norm(error)
    
    if i%interval == 0 and debug:
        print(obj_features)
        print(error, error_norm)

    if np.linalg.norm(error) < 10:
        print ("Goal point achieved!!!")
        break
        
    T = control2robot(wheel_radius, axle_length) # wheel speeds to robot velocities transform
    H = robot2world(orientation)  # robot velocities to world frame transform
    
    # Image Jacobian based on 4 fp of reference object
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
    
    # implement w = gain * pinv(LHT).error
    J = np.dot(L,np.dot(H,T))
    inv_J = np.linalg.pinv(J)
    wheel_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
    robot_velocities = np.dot(T, wheel_velocities)
    w_r = omega2speed(wheel_velocities[0,0],wheel_calibration) 
    w_l = omega2speed(wheel_velocities[1,0],wheel_calibration) 
    
    if i%interval == 0 and debug:
        print("Iteration ", i)
        save_snapshot(image, diag_dir, i)
        print(wheel_velocities, (w_l, w_r))
        print(robot_velocities)
        print("Orientation:", orientation)
        print("Depth:", depth)
    
    if no_motion is False:
        robot.set_motors(w_l, w_r)
        time.sleep(Rtime)
        robot.stop()
    
    # Update robot's orientation
    translation_velocity = robot_velocities[0,0]
    angular_velocity = robot_velocities[1,0]
    orientation += Rtime*angular_velocity
    depth -= Rtime*translation_velocity*math.cos(orientation)
    
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image

image_widget.value = bgr8_to_jpeg(image)
robot.stop()

end = time.perf_counter()
print ("FPS: {:.1f}".format(num_iter/(end-start)))

62 couch [(175, 84), (237, 84), (237, 186), (175, 186)]
laptop 72 [(36, 67), (119, 125)]
couch 62 [(165, 119), (200, 184)]
[(165, 119), (200, 119), (200, 184), (165, 184)]
[[ 10 -35  37 -35  37   2  10   2]] 73.45747068882783
Iteration  0
[[-10.79290285]
 [-12.02663899]] (-0.4484686133429327, -0.4270867045439424)
[[-0.37081755]
 [ 0.33413687]]
Orientation: 0
Depth: 1.3
laptop 72 [(37, 67), (123, 126)]
couch 62 [(164, 119), (207, 183)]
couch 62 [(177, 119), (216, 185)]
laptop 72 [(58, 73), (129, 131)]
[(177, 119), (216, 119), (216, 185), (177, 185)]
[[ -2 -35  21 -35  21   1  -2   1]] 57.810033731178535
Iteration  2
[[-4.88084825]
 [-6.13933353]] (-0.3425526679009986, -0.3191607481166443)
[[-0.17907795]
 [ 0.34083976]]
Orientation: 0.06617213520205803
Depth: 1.3704190123356164
laptop 72 [(73, 78), (138, 133)]
couch 62 [(182, 122), (227, 185)]
couch 62 [(191, 119), (236, 184)]
laptop 72 [(86, 78), (144, 136)]
couch 62 [(0, 104), (54, 186)]
[(191, 119), (236, 119), (236, 184), (191, 184)]

In [243]:
# Use horse as reference object
ref_obj = 18
print(ref_obj,COCO_labels[ref_obj])
Rtime = 0.05
speed = 0.3

# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

for i in range(100):
    rotate_left_2wheels(speed, Rtime)
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=False) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)

    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image
    
    if obj_features is None:
        continue
    else:
        Rtime = 0.01
        speed = 0.29
        norm_distance = np.linalg.norm(focal_center - object_center(obj_features))
        print(COCO_labels[ref_obj], " at ", obj_features)
        print("norm(distance):", norm_distance)
        if  norm_distance < 70:
            print("Reference object near focal center !!!")
            break
        continue
 
    time.sleep(1.0)

18 horse


horse  at  [(0, 93), (81, 93), (81, 204), (0, 204)]
norm(distance): 107.94763831886796
horse  at  [(0, 95), (101, 95), (101, 200), (0, 200)]
norm(distance): 98.92121002957626
horse  at  [(0, 95), (107, 95), (107, 201), (0, 201)]
norm(distance): 95.95022037025906
horse  at  [(20, 100), (120, 100), (120, 198), (20, 198)]
norm(distance): 80.44948792548179
horse  at  [(26, 103), (127, 103), (127, 203), (26, 203)]
norm(distance): 72.86130562851815
horse  at  [(69, 110), (154, 110), (154, 197), (69, 197)]
norm(distance): 43.426848665426306
Reference object near focal center !!!


In [253]:
# Use stool_2 as reference object
ref_obj = 62
print(ref_obj,COCO_labels[ref_obj])
Rtime = 0.05
speed = 0.3

# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

for i in range(3):
    rotate_left_2wheels(speed, Rtime)

for i in range(100):
    rotate_left_2wheels(speed, Rtime)
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=False) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)

    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image
    
    if obj_features is None:
        continue
    else:
        Rtime = 0.01
        speed = 0.29
        norm_distance = np.linalg.norm(focal_center - object_center(obj_features))
        print(COCO_labels[ref_obj], " at ", obj_features)
        print("norm(distance):", norm_distance)
        if  norm_distance < 50:
            print("Reference object near focal center !!!")
            break
        continue
 
    time.sleep(1.0)

62 couch


couch  at  [(226, 68), (297, 68), (297, 197), (226, 197)]
norm(distance): 129.68433209141097
couch  at  [(12, 109), (68, 109), (68, 186), (12, 186)]
norm(distance): 108.75113087281461
couch  at  [(15, 109), (71, 109), (71, 185), (15, 185)]
norm(distance): 106.10256197627851
couch  at  [(22, 111), (78, 111), (78, 185), (22, 185)]
norm(distance): 99.20389279946687
couch  at  [(23, 113), (78, 113), (78, 184), (23, 184)]
norm(distance): 98.55708801140236
couch  at  [(26, 114), (82, 114), (82, 183), (26, 183)]
norm(distance): 95.29969590623294
couch  at  [(28, 110), (82, 110), (82, 184), (28, 184)]
norm(distance): 94.94589002336771
couch  at  [(28, 110), (82, 110), (82, 184), (28, 184)]
norm(distance): 94.94589002336771
couch  at  [(29, 108), (83, 108), (83, 186), (29, 186)]
norm(distance): 94.02554296677076
couch  at  [(34, 111), (85, 111), (85, 185), (34, 185)]
norm(distance): 90.41531383731953
couch  at  [(38, 113), (87, 113), (87, 183), (38, 183)]
norm(distance): 87.67081082036847
couch

In [306]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

# robot pose - may need to be updated
depth = 1.8
orientation = math.pi/2

# Control loop parameters
gain = 1.3
Rtime = 0.1  # The loop run at 8fps --> 0.125s duration
num_iter = 10
interval = 2

no_motion = False  # Flag to disable motor (for debugging)
debug = True

# Use stool as reference object
ref_obj = 62
goal_features = create_feature_pts(mid_bbox_stool_2)
print(ref_obj,COCO_labels[ref_obj], goal_features)

start = time.perf_counter()
# Run for fixed # of iterations
for i in range(num_iter):
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=True) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
    
    if obj_features is None:
        print("Reference object not detected!!!")
        continue
        
    # bound reference object's feature points at goal pose in red
    cv2.rectangle(image, goal_features[0], goal_features[2], RED, 2)
    error = features_error(obj_features,goal_features)
    error_norm = np.linalg.norm(error)
    
    if i%interval == 0 and debug:
        print(obj_features)
        print(error, error_norm)

    if np.linalg.norm(error) < 10:
        print ("Goal point achieved!!!")
        break
        
    T = control2robot(wheel_radius, axle_length) # wheel speeds to robot velocities transform
    H = robot2world(orientation)  # robot velocities to world frame transform
    
    # Image Jacobian based on 4 fp of reference object
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
    
    # implement w = gain * pinv(LHT).error
    J = np.dot(L,np.dot(H,T))
    inv_J = np.linalg.pinv(J)
    wheel_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
    robot_velocities = np.dot(T, wheel_velocities)
    
    if i%interval == 0 and debug:
        print("Iteration ", i)
        save_snapshot(image, diag_dir, i)
        print ()
        print("Wheel speeds:", wheel_velocities, (w_l, w_r))
        print("Robot speeds:", robot_velocities)
        print("Orientation:", orientation)
        print("Depth:", depth)
    
    # limit robot translational speed to 0.9m/s
    tranl_velocity = robot_velocities[0,0]
    if abs(tranl_velocity) > 0.9:
        robot_velocities[0,0] = tranl_velocity/abs(tranl_velocity) * 0.6
        wheel_velocities = np.dot(np.linalg.pinv(T), robot_velocities)
        print("Robot speed clamped.")
        print("Robot speeds:", robot_velocities)
    
    w_r = omega2speed(wheel_velocities[0,0],wheel_calibration) 
    w_l = omega2speed(wheel_velocities[1,0],wheel_calibration) 
    
    if i%interval == 0 and debug:
        print("left wheel:", w_l, " right wheel:", w_r)
   
    if no_motion is False:
        robot.set_motors(w_l, w_r)
        time.sleep(Rtime)
        robot.stop()
    
    # Update robot's orientation
    translation_velocity = robot_velocities[0,0]
    angular_velocity = robot_velocities[1,0]
    orientation += Rtime*angular_velocity
    depth -= Rtime*translation_velocity*math.cos(orientation)
    
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image

image_widget.value = bgr8_to_jpeg(image)
robot.stop()

end = time.perf_counter()
print ("FPS: {:.1f}".format(num_iter/(end-start)))

62 couch [(163, 80), (263, 80), (263, 185), (163, 185)]
couch 62 [(152, 105), (194, 177)]
[(152, 105), (194, 105), (194, 177), (152, 177)]
[[ 11 -25  69 -25  69   8  11   8]] 105.55567251455508
Iteration  0

Wheel speeds: [[6.45787053]
 [8.53331263]] (-0.5300455491019078, -0.5340570150006578)
Robot speeds: [[ 0.24360673]
 [-0.5620989 ]]
Orientation: 1.5707963267948966
Depth: 1.8
left wheel: 0.3870504205742865  right wheel: 0.3484734299624129
couch 62 [(153, 110), (193, 179)]
couch 62 [(149, 105), (189, 181)]
[(149, 105), (189, 105), (189, 181), (149, 181)]
[[ 14 -25  74 -25  74   4  14   4]] 112.36547512470189
Iteration  2

Wheel speeds: [[5.36206818]
 [7.40924428]] (0.4010914536555295, 0.3632349854161815)
Robot speeds: [[ 0.20753383]
 [-0.55444353]]
Orientation: 1.459311123001397
Depth: 1.795640262516808
left wheel: 0.36615695691406197  right wheel: 0.3281053565220173
couch 62 [(145, 102), (188, 178)]
vase 85 [(230, 16), (292, 96)]
couch 62 [(140, 103), (182, 176)]
[(140, 103), (182, 

## QC

In [307]:
obj_features = [(152, 105), (194, 105), (194, 177), (152, 177)]
error = np.array([[ 11, -25,  69, -25,  69,   8,  11,   8]])

orientation = 1.5707963267948966
depth = 1.8

T = control2robot(wheel_radius, axle_length) # wheel speeds to robot velocities transform
H = robot2world(orientation)  # robot velocities to world frame transform

# Image Jacobian based on 4 fp of reference object
UL_fp, UR_fp, LR_fp, LL_fp = obj_features
L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
    
# implement w = gain * pinv(LHT).error
J = np.dot(L,np.dot(H,T))
inv_J = np.linalg.pinv(J)

wheel_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
robot_velocities = np.dot(T, wheel_velocities)

print (robot_velocities)

[[ 0.24360673]
 [-0.5620989 ]]


In [256]:
wheel_velocities = np.array([[-127.14279363], [-124.01202694]])

In [300]:
robot_velocities

array([[ 0.        ],
       [-0.00273215]])

In [299]:
robot_velocities[0,0] = 0

In [303]:
tranl_velocity = robot_velocities[0,0]
if abs(tranl_velocity) > 0.9:
    robot_velocities[0,0] = tranl_velocity/abs(tranl_velocity) * 0.6

In [304]:
robot_velocities

array([[ 0.        ],
       [-0.00273215]])